# Tutorial 05 - Edges

## Dr. David C. Schedl

Note: this tutorial is geared towards students **experienced in programming** and aims to introduce you to **Digital Imaging / Computer Vision** techniques.


# Table of Contents  

- Practice with Filters
- Canny Edge Detection


# Initilization

As always let's import useful libraries, first.

In [ ]:
import os
import cv2 # openCV
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

We will work with images today. So let's download some with `curl` (the same sources as in `02_OpenCV.ipynb`).

In [ ]:
!curl -o "cat.jpg" "https://placekitten.com/320/320" --silent
!curl -o "gogh.jpg" "https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Vincent_van_Gogh_-_National_Gallery_of_Art.JPG/367px-Vincent_van_Gogh_-_National_Gallery_of_Art.JPG" --silent
!curl -o "einstein.jpg" "https://www.cns.nyu.edu/~lcv/ssim/index_files/image003.jpg" --silent
!curl -o "woman.jpg" "https://live.staticflickr.com/8859/18045025168_3a1ffa6521_c_d.jpg" --silent

# Interpolation in Python


## Interpolation in 1D

First let's look at interpolation in 1D. We will use the `scipy.interpolate` module with the `interp1d` function to approximate a function of sines. 

In [ ]:
# 1D Interpolation in Python
from scipy.interpolate import interp1d

x_max = 2.5*np.pi
x = np.linspace(0, x_max, 1000)
fun = lambda x: np.sin(3*x) + np.sin(2*x) + np.sin(x)# + np.sin(0.5*x)

# make the figure larger
plt.figure(figsize=(13, 8))

plt.plot(x, fun(x), 'k:', label='GT')

# only sample 15 points
x_ = np.linspace(0, x_max, 15)
y_ = fun(x_)
plt.plot(x_, y_, 'ko', label='Samples')

for kind in ['nearest', 'linear', 'cubic']:
    f = interp1d(x_, y_, kind=kind)
    plt.plot(x, f(x), '--' if kind == 'nearest' else '', label=kind.capitalize()+" interpolation")


plt.legend(loc='best')
plt.show()

## 2D Image Interpolation using OpenCV

Now let's look at 2D interpolation. We will use the `cv2.resize` function to resize an image to a 10-times larger size. <br>
We will use the `interpolation=` parameter to specify the interpolation methods nearest, bilinear, bicubic and Lanczos. <br>
While cubic interpolation uses 4x4 pixel neighborhood, Lanczos uses 8x8 pixels.

In [ ]:
# 2D Interpolation with OpenCV

# make a synthetic image
img = np.zeros((32,32,3), dtype=np.uint8)
img[0:16, 0:16, 0] = 255
img[np.eye(32, dtype=bool),1] = 255
img[np.roll(np.eye(32, dtype=bool),(1,0)),1] = 255
img[16:19, :, 2] = 255

# load an image
img = cv2.resize(cv2.imread('cat.jpg'), (32, 32), interpolation=cv2.INTER_AREA)


# make the figure larger
plt.figure(figsize=(10, 11))

# resize the image to 256x256 pixels with nearest, bilinear, and bicubic interpolation
for i, kind in enumerate(['nearest', 'bilinear', 'bicubic', 'lanczos4']):
    cv_inter = {
        'nearest': cv2.INTER_NEAREST, 
        'bilinear': cv2.INTER_LINEAR, 
        'bicubic': cv2.INTER_CUBIC,
        'lanczos4': cv2.INTER_LANCZOS4
    }[kind]
    plt.subplot(2, 2, i+1)
    plt.imshow(cv2.resize(img, (320, 320), interpolation=cv_inter))
    plt.axis('off')
    plt.title(kind.capitalize()+" interpolation")


# Practice with linear filters

Image filters in OpenCV are applied with `cv2.filter2D(img,-1,kernel)`, where the image and the kernel are numpy arrays.
You can define a custom kernel by defining a numpy array:  
```
np.array([[0,0,0],
          [0,1,0],
          [0,0,0]])
```


## Exercise 1 📝: <a name="Exercise_1" id="Exercise_1">  </a> What results do you expect?

What results do you expect if you apply the following kernels:

1. 
    \begin{pmatrix}
    0 & 0 & 0\\
    0 & 1 & 0\\
    0 & 0 & 0 
    \end{pmatrix}

2. 
    \begin{pmatrix}
    0 & 0 & 0\\
    0 & 0 & 1\\
    0 & 0 & 0 
    \end{pmatrix}

3. 
    \begin{pmatrix}
    1 & 0 & -1\\
    2 & 0 & -2\\
    1 & 0 & -1 
    \end{pmatrix}

4. Furthermore, what do you expect if you filter an image with:
  \begin{pmatrix}
      0 & 0 & 0\\
      0 & 2 & 0\\
      0 & 0 & 0 
  \end{pmatrix}
  and subtract another filtered image from the result. The second image is filtered with:    
  \begin{pmatrix}
      \frac{1}{9}  & \frac{1}{9}  & \frac{1}{9} \\
      \frac{1}{9}  & \frac{1}{9}  & \frac{1}{9} \\
      \frac{1}{9}  & \frac{1}{9}  & \frac{1}{9} 
  \end{pmatrix} 

In [ ]:
img = cv2.imread('einstein.jpg')[:,:,1].astype(np.float32)

# define your costum kernel below
kernel = np.array([[0,0,0],
                   [0,1,0],
                   [0,0,0]])
dst = cv2.filter2D(img,-1,kernel)

plt.figure(figsize=(15,10)) # this command makes the figure larger so we see the filter results clearer
plt.subplot(121),plt.imshow(img, cmap='gray'),plt.title('Original')
plt.subplot(122),plt.imshow((dst), cmap='gray'),plt.title(r'Filtered ({}$\times${})'.format(*kernel.shape[:2]))
plt.show()

# Canny Edge Detection

Probably the most widely used edge detector in computer vision.
Canny showed that first derivative of Gaussian closely approximates the operator that optimizes the product of signal-to-noise ratio and localization.

Scientific Paper: [J. Canny, A Computational Approach To Edge Detection, IEEE Trans. Pattern Analysis and Machine Intelligence, 8:679-714, 1986.](https://ieeexplore.ieee.org/document/4767851?isnumber=4767846&arnumber=4767851&count=16&index=4)


## OpenCV's Canny implementation

Let's first look at the implementation available with OpenCV. 
```
cv2.Canny(dx, dy, threshold1, threshold2)
```
The `Canny` function takes the derivatives in x and y and two hysteresis thresholds as input.
Let's also look at the magnitude and the orientation.

In [ ]:
img = cv2.imread('woman.jpg',0)

gsize = (15,15)
#img = cv2.GaussianBlur(img, gsize, 3)

ksize = 3
#edges = cv2.Canny(img,100,200,apertureSize=ksize)

sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=ksize)
sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=ksize)

plt.figure(figsize=(20,10))
plt.subplot(1,2,1),plt.imshow(sobelx,cmap = 'gray'), plt.title('Sobel X')
plt.subplot(1,2,2),plt.imshow(sobely,cmap = 'gray'), plt.title('Sobel Y')
plt.show()

mag = np.sqrt( sobelx**2 + sobely**2 )
theta = np.arctan2(sobelx, sobely)

plt.figure(figsize=(20,10)) # this command makes the figure larger so we see the filter results clearer
plt.subplot(121), plt.imshow(mag,cmap = 'gray'), plt.title('Magnitude')
plt.subplot(122), plt.imshow(theta,cmap='hsv'), plt.title('Orientation')
plt.show()

# OpenCVs implementation using the gradients in x and y
edges = cv2.Canny(sobelx.astype(np.int16), sobely.astype(np.int16), 0.1*np.amax(mag),0.2*np.amax(mag))

plt.figure(figsize=(20,10)) # this command makes the figure larger so we see the filter results clearer
plt.subplot(121), plt.imshow(img,cmap = 'gray'), plt.title('Original')
plt.subplot(122), plt.imshow(edges,cmap = 'gray'), plt.title('Canny Edge Image')
plt.show()